<h1> 2. Creating a sampled dataset </h1>

This notebook illustrates:
<ol>
<li> Sampling a BigQuery dataset to create datasets for ML
<li> Preprocessing with Pandas
</ol>

In [5]:
# change these to try this notebook out
BUCKET = 'pipeline-gb'
PROJECT = 'pipeline-gb'
REGION = 'us-west1-b'

In [6]:
import os
os.environ['BUCKET'] = BUCKET
os.environ['PROJECT'] = PROJECT
os.environ['REGION'] = REGION

In [7]:
%%bash
if ! gsutil ls | grep -q gs://${BUCKET}/; then
  gsutil mb -l ${REGION} gs://${BUCKET}
fi

<h2> Create ML dataset by sampling using BigQuery </h2>
<p>
Let's sample the BigQuery data to create smaller datasets.
</p>

In [8]:
# Create SQL query using natality data after the year 2000
from google.cloud import bigquery
query = """
SELECT
  weight_pounds,
  is_male,
  mother_age,
  plurality,
  gestation_weeks,
  FARM_FINGERPRINT(CONCAT(CAST(YEAR AS STRING), CAST(month AS STRING))) AS hashmonth
FROM
  publicdata.samples.natality
WHERE year > 2000
"""

There are only a limited number of years and months in the dataset. Let's see what the hashmonths are.

In [9]:
# Call BigQuery but GROUP BY the hashmonth and see number of records for each group to enable us to get the correct train and evaluation percentages
df = bigquery.Client().query("SELECT hashmonth, COUNT(weight_pounds) AS num_babies FROM (" + query + ") GROUP BY hashmonth").to_dataframe()
print("There are {} unique hashmonths.".format(len(df)))
df.head()

There are 96 unique hashmonths.


,hashmonth,num_babies
0,-1403073183891835564,351299
1,1451354159195218418,334485
2,3545707052733304728,327823
3,2246942437170405963,330315
4,-9183605629983195042,329975


Here's a way to get a well distributed portion of the data in such a way that the test and train sets do not overlap:

In [10]:
# Added the RAND() so that we can now subsample from each of the hashmonths to get approximately the record counts we want
trainQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) < 3 AND RAND() < 0.0005"
evalQuery = "SELECT * FROM (" + query + ") WHERE ABS(MOD(hashmonth, 4)) = 3 AND RAND() < 0.0005"
traindf = bigquery.Client().query(trainQuery).to_dataframe()
evaldf = bigquery.Client().query(evalQuery).to_dataframe()
print("There are {} examples in the train dataset and {} in the eval dataset".format(len(traindf), len(evaldf)))

There are 13330 examples in the train dataset and 3341 in the eval dataset


<h2> Preprocess data using Pandas </h2>
<p>
Let's add extra rows to simulate the lack of ultrasound. In the process, we'll also change the plurality column to be a string.

In [11]:
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.436192,False,29,1,38.0,3095933535584005890
1,6.062712,True,21,1,42.0,3095933535584005890
2,8.999270,True,19,1,38.0,3095933535584005890
3,6.007597,False,25,1,36.0,3095933535584005890
4,6.889446,False,25,1,39.0,3095933535584005890


Also notice that there are some very important numeric fields that are missing in some rows (the count in Pandas doesn't count missing data)

In [12]:
# Let's look at a small sample of the training data
traindf.describe()

,weight_pounds,mother_age,plurality,gestation_weeks,hashmonth
count,13319.000000,13330.000000,13330.000000,13223.000000,1.333000e+04
mean,7.258613,27.262791,1.032483,38.658852,3.064329e+17
std,1.306249,6.121892,0.188366,2.532889,5.196963e+18
min,0.626113,12.000000,1.000000,17.000000,-9.183606e+18
25%,6.587412,22.000000,1.000000,38.000000,-3.340563e+18
50%,7.341393,27.000000,1.000000,39.000000,-3.280124e+17
75%,8.062305,32.000000,1.000000,40.000000,4.331750e+18
max,13.249782,54.000000,5.000000,47.000000,8.599690e+18


In [13]:
# It is always crucial to clean raw data before using in ML, so we have a preprocessing step
import pandas as pd
def preprocess(df):
  # clean up data we don't want to train on
  # in other words, users will have to tell us the mother's age
  # otherwise, our ML service won't work.
  # these were chosen because they are such good predictors
  # and because these are easy enough to collect
  df = df[df.weight_pounds > 0]
  df = df[df.mother_age > 0]
  df = df[df.gestation_weeks > 0]
  df = df[df.plurality > 0]
  
  # modify plurality field to be a string
  twins_etc = dict(zip([1,2,3,4,5],
                   ['Single(1)', 'Twins(2)', 'Triplets(3)', 'Quadruplets(4)', 'Quintuplets(5)']))
  df['plurality'].replace(twins_etc, inplace=True)
  
  # now create extra rows to simulate lack of ultrasound
  nous = df.copy(deep=True)
  nous.loc[nous['plurality'] != 'Single(1)', 'plurality'] = 'Multiple(2+)'
  nous['is_male'] = 'Unknown'
  
  return pd.concat([df, nous])

In [14]:
traindf.head()# Let's see a small sample of the training data now after our preprocessing
traindf = preprocess(traindf)
evaldf = preprocess(evaldf)
traindf.head()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
0,7.436192,False,29,Single(1),38.0,3095933535584005890
1,6.062712,True,21,Single(1),42.0,3095933535584005890
2,8.999270,True,19,Single(1),38.0,3095933535584005890
3,6.007597,False,25,Single(1),36.0,3095933535584005890
4,6.889446,False,25,Single(1),39.0,3095933535584005890


In [15]:
traindf.tail()

,weight_pounds,is_male,mother_age,plurality,gestation_weeks,hashmonth
13325,9.312326,Unknown,34,Single(1),39.0,-774501970389208065
13326,7.936641,Unknown,19,Single(1),39.0,-774501970389208065
13327,8.666372,Unknown,25,Single(1),39.0,-774501970389208065
13328,8.811877,Unknown,36,Single(1),39.0,-774501970389208065
13329,6.999677,Unknown,31,Multiple(2+),35.0,-774501970389208065


In [16]:
# Describe only does numeric columns, so you won't see plurality
traindf.describe()

,weight_pounds,mother_age,gestation_weeks,hashmonth
count,26432.000000,26432.000000,26432.000000,2.643200e+04
mean,7.259153,27.253481,38.666314,3.036259e+17
std,1.304331,6.121831,2.506493,5.192778e+18
min,0.626113,12.000000,17.000000,-9.183606e+18
25%,6.578594,22.000000,38.000000,-3.340563e+18
50%,7.341393,27.000000,39.000000,-3.280124e+17
75%,8.062305,32.000000,40.000000,4.331750e+18
max,13.249782,54.000000,47.000000,8.599690e+18


<h2> Write out </h2>
<p>
In the final versions, we want to read from files, not Pandas dataframes. So, write the Pandas dataframes out as CSV files. 
Using CSV files gives us the advantage of shuffling during read. This is important for distributed training because some workers might be slower than others, and shuffling the data helps prevent the same data from being assigned to the slow workers.


In [17]:
traindf.to_csv('train.csv', index=False, header=False)
evaldf.to_csv('eval.csv', index=False, header=False)

In [18]:
%%bash
wc -l *.csv
head *.csv
tail *.csv

   6646 eval.csv
  26432 train.csv
  33078 total
==> eval.csv <==
5.8312268299,False,31,Single(1),36.0,-4740473290291881219
6.87621795178,False,25,Single(1),41.0,-1639186255933990135
7.87491199864,False,40,Single(1),37.0,1569531340167098963
7.936641432,False,27,Single(1),38.0,-7146494315947640619
6.93794738514,True,29,Single(1),37.0,270792696282171059
8.6421206704,True,31,Single(1),40.0,270792696282171059
7.7602716223999995,True,37,Single(1),39.0,3182182455926341111
6.3118345610599995,False,22,Single(1),38.0,-6141045177192779423
7.68751907594,False,33,Single(1),35.0,8904940584331855459
8.24969784404,False,28,Single(1),43.0,-1866590652208008467

==> train.csv <==
7.43619209726,False,29,Single(1),38.0,3095933535584005890
6.0627122049999995,True,21,Single(1),42.0,3095933535584005890
8.99926953484,True,19,Single(1),38.0,3095933535584005890
6.0075966395,False,25,Single(1),36.0,3095933535584005890
6.8894456875,False,25,Single(1),39.0,3095933535584005890
4.37397127808,True,39,Twins(2),37.0,30

Copyright 2017-2018 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License